In [1]:
!pip install catboost
!pip install xgboost
!pip install openml
!pip install numba

In [2]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load balance-scale dataset
dataset = openml.datasets.get_dataset(11, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 400
Validation set size: 100
Test set size: 125


,left-weight,left-distance,right-weight,right-distance
538,5.0,2.0,3.0,4.0
161,2.0,2.0,3.0,2.0
541,5.0,2.0,4.0,2.0
233,2.0,5.0,2.0,4.0
232,2.0,5.0,2.0,3.0


In [4]:
from GRANDE import GRANDE

params = {
        'depth': 5, # tree depth
        'n_estimators': 2048, # number of estimators / trees

        'learning_rate_weights': 0.005, # learning rate for leaf weights
        'learning_rate_index': 0.01, # learning rate for split indices
        'learning_rate_values': 0.01, # learning rate for split values
        'learning_rate_leaf': 0.01, # learning rate for leafs (logits)

        'optimizer': 'adam', # optimizer
        'cosine_decay_steps': 0, # decay steps for lr schedule (CosineDecayRestarts)

        'loss': 'crossentropy', # loss function (default 'crossentropy' for binary & multi-class classification and 'mse' for regression)
        'focal_loss': False, # use focal loss {True, False}
        'temperature': 0.0, # temperature for stochastic re-weighted GD (0.0, 1.0)

        'from_logits': True, # use logits for weighting {True, False}
        'use_class_weights': True, # use class weights for training {True, False}

        'dropout': 0.0, # dropout rate (here, dropout randomly disables individual estimators of the ensemble during training)

        'selected_variables': 0.8, # feature subset percentage (0.0, 1.0)
        'data_subset_fraction': 1.0, # data subset percentage (0.0, 1.0)
}

args = {
    'epochs': 1_000, # number of epochs for training
    'early_stopping_epochs': 25, # patience for early stopping (best weights are restored)
    'batch_size': 64,  # batch size for training

    'cat_idx': categorical_feature_indices, # put list of categorical indices
    'objective': 'classification', # objective / task {'binary', 'classification', 'regression'}
    
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2024-03-14 12:10:04.523997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 12:10:11.217106: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-14 12:10:18.890973: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-14 12:10:18.891464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46090 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6


Epoch 1/1000


I0000 00:00:1710414622.260407  229213 service.cc:145] XLA service 0x7f854c004b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710414622.260449  229213 service.cc:153]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-14 12:10:22.322273: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-14 12:10:22.550114: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


1/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - loss: 1.2052

I0000 00:00:1710414623.543500  229213 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 340ms/step - loss: 1.1278 - val_loss: 1.0527
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0825 - val_loss: 1.0108
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0406 - val_loss: 0.9691
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9993 - val_loss: 0.9278
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9588 - val_loss: 0.8876
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9199 - val_loss: 0.8495
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8831 - val_loss: 0.8125
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8477 - val_loss: 0.7776
Epoch 9/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8137 - val_loss: 0.7451
Epoch 10/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7808 - val_loss: 0.7128
Epoch 11/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7470 - val_loss: 0.6839
Epoch 12/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7156 - val_loss: 0.6542


In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)
    
encoder = ce.OrdinalEncoder(cols=X_train.columns[categorical_feature_indices])
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [7]:
low_cardinality_indices = []
high_cardinality_indices = []

categorical_feature_indices = []
for column_index in range(X_train.shape[1]):
    if categorical_indicator[column_index]:
        categorical_feature_indices.append(column_index)
        if len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(X_train.columns[column_index])
        else:
            high_cardinality_indices.append(X_train.columns[column_index])

In [8]:
encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

quantile_noise = 1e-4
quantile_train = np.copy(X_train.values).astype(np.float64)
np.random.seed(42)
stds = np.std(quantile_train, axis=0, keepdims=True)
noise_std = quantile_noise / np.maximum(stds, quantile_noise)
quantile_train += noise_std * np.random.randn(*quantile_train.shape)       

scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
scaler.fit(quantile_train)

X_train = scaler.transform(X_train.values.astype(np.float64))
X_valid = scaler.transform(X_valid.values.astype(np.float64))
X_test = scaler.transform(X_test.values.astype(np.float64))

/home/smarton/anaconda3/envs/GradientBasedTreeLearning/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (400). n_quantiles is set to n_samples.
  warnings.warn(


In [9]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )
if args['objective'] == 'regression':
    preds_xgb = model_xgb.predict(X_test)
else:
    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-mlogloss:0.92906
[1]	validation_0-mlogloss:0.84543
[2]	validation_0-mlogloss:0.77895
[3]	validation_0-mlogloss:0.74425
[4]	validation_0-mlogloss:0.73254
[5]	validation_0-mlogloss:0.71118
[6]	validation_0-mlogloss:0.70206
[7]	validation_0-mlogloss:0.69809
[8]	validation_0-mlogloss:0.70716
[9]	validation_0-mlogloss:0.71560
[10]	validation_0-mlogloss:0.72703
[11]	validation_0-mlogloss:0.72930
[12]	validation_0-mlogloss:0.73781
[13]	validation_0-mlogloss:0.74436
[14]	validation_0-mlogloss:0.75612
[15]	validation_0-mlogloss:0.76647
[16]	validation_0-mlogloss:0.77294
[17]	validation_0-mlogloss:0.76826
[18]	validation_0-mlogloss:0.77106
[19]	validation_0-mlogloss:0.78535
[20]	validation_0-mlogloss:0.78614
[21]	validation_0-mlogloss:0.79861
[22]	validation_0-mlogloss:0.80108
[23]	validation_0-mlogloss:0.81176
[24]	validation_0-mlogloss:0.81198
[25]	validation_0-mlogloss:0.81683
[26]	validation_0-mlogloss:0.82233
[27]	validation_0-mlogloss:0.83074


In [10]:
from catboost import CatBoostClassifier, Pool

model_catboost = CatBoostClassifier(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

if args['objective'] == 'regression':
    preds_catboost = model_catboost.predict(X_test_raw)
else:
    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.106015
0:	learn: 1.0490346	test: 1.0632694	best: 1.0632694 (0)	total: 47.7ms	remaining: 47.6s
1:	learn: 0.9981136	test: 1.0086553	best: 1.0086553 (1)	total: 49.3ms	remaining: 24.6s
2:	learn: 0.9493243	test: 0.9615261	best: 0.9615261 (2)	total: 51ms	remaining: 16.9s
3:	learn: 0.9089973	test: 0.9258129	best: 0.9258129 (3)	total: 52.5ms	remaining: 13.1s
4:	learn: 0.8773156	test: 0.9026201	best: 0.9026201 (4)	total: 54.2ms	remaining: 10.8s
5:	learn: 0.8428892	test: 0.8817201	best: 0.8817201 (5)	total: 55.8ms	remaining: 9.24s
6:	learn: 0.8179774	test: 0.8639109	best: 0.8639109 (6)	total: 56.9ms	remaining: 8.07s
7:	learn: 0.7867788	test: 0.8385414	best: 0.8385414 (7)	total: 58.3ms	remaining: 7.23s
8:	learn: 0.7617402	test: 0.8156207	best: 0.8156207 (8)	total: 59.5ms	remaining: 6.55s
9:	learn: 0.7363277	test: 0.7937842	best: 0.7937842 (9)	total: 61ms	remaining: 6.04s
10:	learn: 0.7159307	test: 0.7786301	best: 0.7786301 (10)	total: 62.3ms	remaining: 5.6s
11:	learn: 0.700

In [11]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande))

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb))

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost))

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

Accuracy GRANDE: 0.896
F1 Score GRANDE: 0.7093081509667817
ROC AUC GRANDE: 0.928491385348088


Accuracy XGB: 0.768
F1 Score XGB: 0.6161896745230079
ROC AUC XGB: 0.7848764999766541


Accuracy CatBoost: 0.792
F1 Score CatBoost: 0.6638439468628149
ROC AUC CatBoost: 0.866746042863146




In [12]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()